In [ ]:
# Integrating Raster Data stored in Postgis (Raster)

The raster_api module imports raster data stored in the postgis Database. It can be exported into numpy narrays, gdal files or summary statistics.
The intention is to use it together with a grid and generate summary statistics.

## The aggregates
These are functions that aggregate the raster data to fit a given geometry (a closed polygon). 
The aggregates can be:
    * Union (returning the original raster data, cliped be the geometry) 
    * Slope (performs the slope algorithm implemented in postgis. For use with DEM models)
    * Aspect (Calculates the aspect (orientation) of pixels)
    * Hillshade (calculate a hillshade image with standard parameters as in Postgis)
    * SummaryStatistics (calculates the vector: {count,sum,mean,stddev,min,max} of the raster data defined by the geometry imposed)
    
The available aggregates are stored in raster_api.aggregates in the dictionary: aggregates_dict

## The lookups
These are functions that are translated in SQL under the WHERE clause. 
The only lookup at this moment is: 
    * intersectWith
And this was done because the current implementation of Raster lookups in Django/Postgis is highly unefficient.
For importing and *registering* the lookup use:


For using this lookup in a filter use:

    [model].filter(rast__intersect_with=other.geometry)

## The *RasterData* class
This class works as a wrapper of Postgis raster models. It provides an interface for processing and analysing raster data stored in a postgis database
For instantiating a particular raster model (data like: DEM, Temperature, etc. ) use:

    * RasterData(RASTERMODEL, geometry)
    e.g.
    x = RasterData(DemMexLow,t1.biomeGeometry)


Remember that the data in Postgis RAster is stored in blobs per row. This means that each blob is a tile of n*n cells in binary format.
That is why there is a highlevel postprocessing afterwards, to integrate the aggregates as methods as well as geoTif exporting and Neo4J. 


In the example above, x is now defined as the elevation data with in the t1 geometry. 
We can export this data as a gdal GeoTif image with the following command.
### First load the desired Raster data (layer)
    x.getRaster()
This will define the x.rasterdata attribute.
This can be accesed as:
    x.rasterdata.bands[0].data()
For visualizing:
    import matplotlib.pyplot as plt
    plt.imshow(x.rasterdata.bands[0].data(),clim=(10,200))
clim is a parameter for visualising the data.

    

<img src="DEM1.png",width=300,height=300>The clip of a DEM within the t0 taxonomy geometry

For exporting the GeoTiff use:
    * x.exportToGeoTiff(filename, path=DEFAULT_PATH_IN_settings.py_FILE)

## Calculating SummaryStatististics.
Once the RasterData has been selected. We can perform a summary stats within the geometry given as argument.
        * x.getSummaryStats()


Returns:
    {'count': 28561,
 'max': 950.0,
 'mean': 290.567942298939,
 'min': 87.0,
 'stddev': 130.578128109693,
 'sum': 8298911.0,
 'uniqueid': 'POLYGON ((-109 27.1875, -108.8125 27.1875, -108.8125 27.375, -109 27.375, -109 27.1875))'}


## Pixel Accessors
The aggregate funtion 'getValue' extracts the pixel value in the intersection with it's interior and the coordinate (point) given.

Let's asume that oc is an occurrence object from a taxonomy t. oc.geom is the geometry (type point).
Now, for getting the raster value in that point use:

    z = RasterData(RasterModel,t.biomegeometry).getValue(oc.geom)